**This notebook is an exercise in the [Data Cleaning](https://www.kaggle.com/learn/data-cleaning) course.  You can reference the tutorial at [this link](https://www.kaggle.com/alexisbcook/inconsistent-data-entry).**

---


In this exercise, you'll apply what you learned in the **Inconsistent data entry** tutorial.

# Setup

The questions below will give you feedback on your work. Run the following cell to set up the feedback system.

In [1]:
from learntools.core import binder
binder.bind(globals())
from learntools.data_cleaning.ex5 import *
print("Setup Complete")

Setup Complete


# Get our environment set up

The first thing we'll need to do is load in the libraries and dataset we'll be using.  We use the same dataset from the tutorial.

In [2]:
# modules we'll use
import pandas as pd
import numpy as np

# helpful modules
import fuzzywuzzy
from fuzzywuzzy import process
import chardet

# read in all our data
professors = pd.read_csv("../input/pakistan-intellectual-capital/pakistan_intellectual_capital.csv")

# set seed for reproducibility
np.random.seed(0)

Next, we'll redo all of the work that we did in the tutorial.

In [3]:
professors.sample(5)

,Unnamed: 0,S#,Teacher Name,University Currently Teaching,Department,Province University Located,Designation,Terminal Degree,Graduated from,Country,Year,Area of Specialization/Research Interests,Other Information
108,300,301,Hareem-e-Sahar,FAST-NU(Islamabad),Computer Science,Capital,Lecturer,BS,FAST– National University of Computer and Emer...,Pakistan,2014.0,NaN,NaN
661,1247,1248,IMTIAZ ALI KOREJO,University of Sindh,Institute of Mathematics & Computer Science,Sindh,Associate Professor,PhD,University of Leicester,UK,2012.0,NaN,NaN
397,821,822,Amna Shahnawaz,University of Central Punjab,Computer Science,Punjab,Associate Professor,MS,Lahore University of Management Sciences,Pakistan,2017.0,NaN,NaN
1006,1695,1696,Aden Iqbal,NFC Institute of Engineering and Fertilizer Re...,Computer Science,Punjab,Lecturer,MS,National Textile University,Pakistan,NaN,NaN,NaN
240,468,469,Mr. Syed Awais Haider,Capital University of Science and Technology,Computer Science,Capital,Lecturer,Mphil,Quaid-i-Azam University,Pakistan,2012.0,NaN,NaN


In [4]:
# convert to lower case
professors['Country'] = professors['Country'].str.lower()
# remove trailing white spaces
professors['Country'] = professors['Country'].str.strip()

# get the top 10 closest matches to "south korea"
countries = professors['Country'].unique()
matches = fuzzywuzzy.process.extract("south korea", countries, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

def replace_matches_in_column(df, column, string_to_match, min_ratio = 47):
    # get a list of unique strings
    strings = df[column].unique()
    
    # get the top 10 closest matches to our input string
    matches = fuzzywuzzy.process.extract(string_to_match, strings, 
                                         limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

    # only get matches with a ratio > 90
    close_matches = [matches[0] for matches in matches if matches[1] >= min_ratio]

    # get the rows of all the close matches in our dataframe
    rows_with_matches = df[column].isin(close_matches)

    # replace all rows with close matches with the input matches 
    df.loc[rows_with_matches, column] = string_to_match
    
    # let us know the function's done
    print("All done!")
    
replace_matches_in_column(df=professors, column='Country', string_to_match="south korea")
countries = professors['Country'].unique()

All done!


# 1) Examine another column

Write code below to take a look at all the unique values in the "Graduated from" column.

In [5]:
# TODO: Your code here
graduated_from = professors['Graduated from'].unique()

Do you notice any inconsistencies in the data?  Can any of the inconsistencies in the data be fixed by removing white spaces at the beginning and end of cells?

Once you have answered these questions, run the code cell below to get credit for your work.

In [6]:
# Check your answer (Run this code cell to receive credit!)
q1.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct:</span> 

There are inconsistencies that can be fixed by removing white spaces at the beginning and end of cells.  For instance, "University of Central Florida" and " University of Central Florida" both appear in the column.

In [7]:
# Line below will give you a hint
q1.hint()

<IPython.core.display.Javascript object>

<span style="color:#3366cc">Hint:</span> Use `unis = professors['Graduated from'].unique()` to take a look at the unique values in the 'Graduated from' column. You may find it useful to sort the data before printing it.

# 2) Do some text pre-processing

Convert every entry in the "Graduated from" column in the `professors` DataFrame to remove white spaces at the beginning and end of cells.

In [8]:
# TODO: Your code here
professors['Graduated from'] = professors['Graduated from'].str.strip()
# Check your answer
q2.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [9]:
# Lines below will give you a hint or solution code
q2.hint()
q2.solution()

<IPython.core.display.Javascript object>

<span style="color:#3366cc">Hint:</span> In the tutorial, you did the same operation on a different column.

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python
professors['Graduated from'] = professors['Graduated from'].str.strip()

```

# 3) Continue working with countries

In the tutorial, we focused on cleaning up inconsistencies in the "Country" column.  Run the code cell below to view the list of unique values that we ended with.

In [10]:
# get all the unique values in the 'City' column
countries = professors['Country'].unique()

# sort them alphabetically and then take a closer look
countries.sort()
countries

array(['australia', 'austria', 'canada', 'china', 'finland', 'france',
       'germany', 'greece', 'hongkong', 'ireland', 'italy', 'japan',
       'macau', 'malaysia', 'mauritius', 'netherland', 'new zealand',
       'norway', 'pakistan', 'portugal', 'russian federation',
       'saudi arabia', 'scotland', 'singapore', 'south korea', 'spain',
       'sweden', 'thailand', 'turkey', 'uk', 'urbana', 'usa', 'usofa'],
      dtype=object)

Take another look at the "Country" column and see if there's any more data cleaning we need to do.

It looks like 'usa' and 'usofa' should be the same country.  Correct the "Country" column in the dataframe to replace 'usofa' with 'usa'.

**Use the most recent version of the DataFrame (with the whitespaces at the beginning and end of cells removed) from question 2.**

In [11]:
# TODO: Your code here!
rows_matches = professors['Country'].isin(['usofa'])
professors.loc[rows_matches,'Country'] = 'usa'
# Check your answer
q3.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [12]:
countries = professors['Country'].unique()
countries.sort()
countries

array(['australia', 'austria', 'canada', 'china', 'finland', 'france',
       'germany', 'greece', 'hongkong', 'ireland', 'italy', 'japan',
       'macau', 'malaysia', 'mauritius', 'netherland', 'new zealand',
       'norway', 'pakistan', 'portugal', 'russian federation',
       'saudi arabia', 'scotland', 'singapore', 'south korea', 'spain',
       'sweden', 'thailand', 'turkey', 'uk', 'urbana', 'usa'],
      dtype=object)

In [13]:
# Lines below will give you a hint or solution code
#q3.hint()
q3.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python
matches = fuzzywuzzy.process.extract("usa", countries, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)
replace_matches_in_column(df=professors, column='Country', string_to_match="usa", min_ratio=70)

```

# Congratulations!

Congratulations for completing the **Data Cleaning** course on Kaggle Learn!

To practice your new skills, you're encouraged to download and investigate some of [Kaggle's Datasets](https://www.kaggle.com/datasets).

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/data-cleaning/discussion) to chat with other learners.*